In [6]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import community as community_louvain

In [2]:
state_bipartite_weighted_network = pd.read_csv("./all_candidates_state_bipartite_weighted_network.csv")
state_top100_contributors_network = pd.read_csv("./state_contributor_top100_contributors_network.csv")
federal_top100_contributors_network = pd.read_csv("./federal_contributor_top100_contributors_network.csv")
winning_candidates_state_bipartite_weighted_network = pd.read_csv("./winning_candidates_state_bipartite_weighted_network.csv")

In [3]:
print("state_bipartite_weighted_network :", state_bipartite_weighted_network.columns)
print("state_top100_contributors_network :", state_top100_contributors_network.columns)
print("federal_top100_contributors_network :", federal_top100_contributors_network.columns)
print("winning_candidates_state_bipartite_weighted_network :", winning_candidates_state_bipartite_weighted_network.columns)

state_bipartite_weighted_network : Index(['Unnamed: 0', 'Unnamed: 1', 'ZZ', 'WA', 'WV', 'DE', 'DC', 'WI', 'NO',
       'HI', 'FL', 'WY', 'NH', 'NJ', 'NM', 'TX', 'LA', 'NB', 'NC', 'ND', 'NE',
       'TN', 'NY', 'PA', 'CA', 'NV', 'VA', 'CH', 'CO', 'ON', 'VI', 'AK', 'AL',
       'BI', 'VQ', 'XX', 'AR', 'VT', 'IL', 'GA', 'IN', 'IA', 'OK', 'AZ', 'ID',
       'CT', 'ME', 'MD', 'MA', 'MC', 'OH', 'UT', 'MO', 'MN', 'MI', 'RI', 'KS',
       'MT', 'UN', 'SN', 'MS', 'PR', 'SC', 'KY', 'OR', 'SD'],
      dtype='object')
state_top100_contributors_network : Index(['Unnamed: 0', 'MCGUIRE, PERRY J DOUGLASVILLE 30135 GA',
       'HANCE, KENT R AUSTIN 78701 TX',
       'PHILADELPHIA TRIAL LAWYERS ASSOCIATION PHILADELPHIA 19107 PA',
       'STRICKLAND, DEANNA L BROOKLET 30415 GA',
       'LAW OFFICES OF PETER G ANGELOS BALTIMORE 21201 VA',
       'ELGIN, ROBERT L UNIVERSITY CITY 63130 MO',
       'BRISBANE, CHARLES LOCUST VALLEY 11560 NY',
       'KRAMER, ORIN S NEW YORK 10022 NY', 'FERNANDEZ, RAUL J RESTO

In [10]:

def extract_features(G, weight_attr='weight'):
    # 1) Degree (edge count)
    degree     = dict(G.degree())
    # 2) Weighted degree (sum of weights)
    wdegree    = dict(G.degree(weight=weight_attr))
    # 3) PageRank (with weights)
    pagerank   = nx.pagerank(G, alpha=0.85, weight=weight_attr, tol=1e-3)
    # 4) Louvain‐style community via NetworkX greedy modularity
    #    (returns a list of sets of nodes)
    communities = nx_community.greedy_modularity_communities(G, weight=weight_attr)
    # flatten to node→community_id
    mod_class = {}
    for cid, com in enumerate(communities):
        for node in com:
            mod_class[node] = cid

    # 5) Construct a DataFrame
    df = pd.DataFrame({
        'degree':           pd.Series(degree),
        'weighted_degree':  pd.Series(wdegree),
        'pagerank':         pd.Series(pagerank),
        'community':        pd.Series(mod_class)
    })
    return df

In [12]:
# 1. federal network
from networkx.algorithms import bipartite, community as nx_community

df_fed = pd.read_csv('./federal_contributor_top100_contributors_network.csv', index_col=0)
G_fed = nx.from_pandas_adjacency(df_fed, create_using=nx.Graph())
features_fed = extract_features(G_fed)
print("Federal top 5 by weighted_degree:\n", features_fed['weighted_degree'].nlargest(5))
features_fed.to_csv('features_fed.csv')

Federal top 5 by weighted_degree:
 HONEYWELL INTERNATIONAL WASHINGTON 20001 DC                     1.236248e+10
CREDIT UNION NATIONAL ASSOCIATION / CUNA WASHINGTON 20004 DC    1.204402e+10
VERIZON COMMUNICATIONS WASHINGTON 20005 DC                      1.091501e+10
UPS ATLANTA 30328 GA                                            1.053287e+10
AMERICAN FAMILY LIFE ASSURANCE CO / AFLAC COLUMBUS 31999 GA     1.051659e+10
Name: weighted_degree, dtype: float64


In [13]:
# 2. state top‑100 network:
df_state = pd.read_csv('./state_contributor_top100_contributors_network.csv', index_col=0)
G_state = nx.from_pandas_adjacency(df_state, create_using=nx.Graph())
features_state = extract_features(G_state)
print("State top 5 by weighted_degree:\n", features_state['weighted_degree'].nlargest(5))
features_state.to_csv('features_state.csv')


State top 5 by weighted_degree:
 NEW JERSEY STATE LABORERS MONROE TOWNSHIP 8831 NJ         4.427149e+08
ELECTRICAL WORKERS LOCAL 164 PARAMUS 7652 NJ              4.095803e+08
CORZINE, JON S HOBOKEN 7030 NJ                            3.761866e+08
1199 SEIU UNITED HEALTH CARE WORKERS NEW YORK 10108 NY    3.099182e+08
NEW JERSEY EDUCATION ASSOCIATION TRENTON 8607 NJ          3.066865e+08
Name: weighted_degree, dtype: float64


In [14]:
# 3. bipartite & projections:
df_all = pd.read_csv('./all_candidates_state_bipartite_weighted_network.csv', index_col=0)
G_state = nx.from_pandas_adjacency(df_state, create_using=nx.Graph())
features_state = extract_features(G_state)
print("State all Candidates top 5 by weighted_degree:\n", features_state['weighted_degree'].nlargest(5))
features_state.to_csv('features_AllCandidates_state.csv')

State all Candidates top 5 by weighted_degree:
 NEW JERSEY STATE LABORERS MONROE TOWNSHIP 8831 NJ         4.427149e+08
ELECTRICAL WORKERS LOCAL 164 PARAMUS 7652 NJ              4.095803e+08
CORZINE, JON S HOBOKEN 7030 NJ                            3.761866e+08
1199 SEIU UNITED HEALTH CARE WORKERS NEW YORK 10108 NY    3.099182e+08
NEW JERSEY EDUCATION ASSOCIATION TRENTON 8607 NJ          3.066865e+08
Name: weighted_degree, dtype: float64


In [15]:
# 4. federal network
from networkx.algorithms import bipartite, community as nx_community

df_fed = pd.read_csv('./winning_candidates_state_bipartite_weighted_network.csv', index_col=0)
G_fed = nx.from_pandas_adjacency(df_fed, create_using=nx.Graph())
features_fed = extract_features(G_fed)
print("Winning top 5 by weighted_degree:\n", features_fed['weighted_degree'].nlargest(5))
features_fed.to_csv('features_WinningCandidates.csv')

NetworkXError: ('Columns must match Indices.', '[\'COLLINS, JEFFREY LYNN\', \'WICKER, ROGER F\', \'CALFEE, KENT\', \'ENZI, MICHAEL BRADLEY (MIKE)\', \'WILSON, BILL\', \'SLOAN, TOM\', \'STEINBORN, JEFF\', \'STARR, TERRELL\', \'GREEN, GENE\', \'HARRISON, JAMES HODGES\', \'SWITALSKI, MICHAEL\', \'DIERKS, CAP\', \'RUST, NANCY\', \'COVEY, JAMES E\', \'APPLE, PAT\', \'TIPTON, SCOTT R\', \'RILEY JR, ROBERT R\', \'HENRY, BRAD\', \'DICKENSON, SUE\', \'COCHRANE, BETSY L\', \'GELSER, SARA A\', \'BASKERVILLE, VIOLA OSBORNE\', \'GILL, NIA H\', \'PEREZ, JOHN A\', \'AGNEW, PAUL L\', \'TANNER, GARY\', \'SANTOS, SHARON TOMIKO\', \'GRANT, JAMES\', \'JUDGE, PATTY\', \'HOPE, MICHAEL S (MIKE)\', \'PEACE, CHRISTOPHER K\', \'MCCOLLUM, BILL\', \'BATTLE JR, JAMES A\', \'PAYNE, GREG ALAN\', \'FORSYTHE, JIM\', \'WILLIAMS, MICHEAL R\', \'SPEARMAN, PATRICIA (PAT)\', \'WHITE, BRIAN KEITH\', \'ZIEMAN, MARK\', \'WILMER, FRANKE\', \'MITCHELL, JOSEPH C\', \'PESQUIERA, CHARLENE\', \'BUCHER, SUSAN\', \'DAYTON, MARK & SOL

In [19]:
import pandas as pd
import networkx as nx
from networkx.algorithms import community as nx_comm

# --- adjust these to your actual file paths ---
files = {
    "all_candidates_state":  "all_candidates_state_bipartite_weighted_network.csv",
    "winning_candidates":    "winning_candidates_state_bipartite_weighted_network.csv",
    "state_top100":          "state_contributor_top100_contributors_network.csv",
    "federal_top100":        "federal_contributor_top100_contributors_network.csv",
}

def analyze_bipartite(path, node_col, prefix):
    df = pd.read_csv(path)
    state_cols = df.columns[2:].tolist()
    G = nx.Graph()
    # add state nodes (cast to str)
    for s in state_cols:
        G.add_node(str(s), type="state")
    # add candidate nodes & edges
    for _, row in df.iterrows():
        cand = str(row[node_col])
        G.add_node(cand, type="candidate")
        for s in state_cols:
            w = row[s]
            if pd.notna(w) and float(w) > 0:
                G.add_edge(cand, str(s), weight=float(w))
    return analyze_graph(G, prefix)

def analyze_onemode(path, prefix):
    df = pd.read_csv(path, index_col=0)
    G = nx.Graph()
    # add nodes (all as str)
    for u in df.index:
        G.add_node(str(u), type="contributor")
    # add edges
    for u in df.index:
        for v, w in df.loc[u].items():
            if pd.notna(w) and float(w) > 0:
                G.add_edge(str(u), str(v), weight=float(w))
    return analyze_graph(G, prefix)

def analyze_graph(G, prefix):
    # 1) degree metrics
    deg  = dict(G.degree())
    wdeg = dict(G.degree(weight="weight"))
    # 2) PageRank (undirected, weighted)
    pr = nx.pagerank(G, alpha=0.85, weight="weight", tol=1e-3)
    # 3) greedy modularity communities (all node IDs are str, so no comparison error)
    communities = nx_comm.greedy_modularity_communities(G, weight="weight")
    comm_map = {}
    for cid, comm in enumerate(communities):
        for node in comm:
            comm_map[node] = cid
    # 4) assemble feature table
    rows = []
    for n in G.nodes:
        rows.append({
            "node":            n,
            "type":            G.nodes[n].get("type",""),
            "degree":          deg.get(n, 0),
            "weighted_degree": wdeg.get(n, 0),
            "pagerank":        pr.get(n, 0),
            "community":       comm_map.get(n, -1),
        })
    feat = pd.DataFrame(rows)
    feat.to_csv(f"{prefix}_features.csv", index=False)
    print(f"→ wrote {prefix}_features.csv ({len(feat)} rows)")
    return feat

# run for all four networks
features = {}
features["all_candidates_state"] = analyze_bipartite(
    files["all_candidates_state"], "Unnamed: 1", "all_candidates_state")
features["winning_candidates"]   = analyze_bipartite(
    files["winning_candidates"],   "Unnamed: 1", "winning_candidates_state")
features["state_top100"]         = analyze_onemode(
    files["state_top100"], "state_top100")
features["federal_top100"]       = analyze_onemode(
    files["federal_top100"], "federal_top100")

# quick peek
print(features["federal_top100"].head())


→ wrote all_candidates_state_features.csv (65 rows)
→ wrote winning_candidates_state_features.csv (65 rows)
→ wrote state_top100_features.csv (200 rows)
→ wrote federal_top100_features.csv (200 rows)
                                                node         type  degree  \
0                     CARVIN, JOE RYE BROOK 10573 NY  contributor       1   
1         MILLER-MEEKS, MARIANNETTE OTTUMWA 52501 IA  contributor      42   
2                    NORCROSS, DONALD CAMDEN 8102 NJ  contributor      46   
3  NATIONAL ASSOCIATION OF REALTORS CHICAGO 60611 IL  contributor     180   
4                 STERN, DIANNE B SCARSDALE 10583 NY  contributor     127   

   weighted_degree  pagerank  community  
0     1.424610e+03  0.000764          0  
1     7.864618e+06  0.000776          0  
2     3.727352e+07  0.000808          1  
3     1.041528e+10  0.014227          1  
4     6.574629e+08  0.001759          1  
